In [17]:
# Assuming `model` is your trained LSTM model
model.eval()  # Set the model to evaluation mode
# example_input = torch.randn(1, 80, 6)  # Adjust based on your input shape
# scripted_model = torch.jit.trace(model, example_input)  # Trace the model with example input
# scripted_model.save("lstm_model_mobile.pt")  # Save the scripted model
torch.save(model.state_dict(), 'LSTM_1CLASS_2DS_200wnd_13_pch_10steps.pth')
print('Model weights saved to step_lstm_more_stairsup.pth')

Model weights saved to step_lstm_more_stairsup.pth


In [9]:
model = LSTM(6, 64, 2, 0.2)
model.load_state_dict(torch.load('LSTM_1CLASS_2DS_200wnd_13_pch_3-3-3-fullset.pth'))
model.eval()  # Set the model to evaluation mode
print('Model weights loaded from model_weights.pth')

Model weights loaded from model_weights.pth


C:\Users\micha\AppData\Local\Temp\ipykernel_27308\3425704084.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('LSTM_1CLASS_2DS_200wnd_13_

# SINGLE VERSION

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, 1)  # Binary classification

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Take the output of the last timestep
        out = self.fc(out)  # Logits
        return out

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import glob

import pandas as pd
import os

class StepDataset(Dataset):
    def __init__(self, folder_path, window_size):
        self.window_size = window_size
        self.data = self.load_data(folder_path)
        self.features = self.data[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']].values
        self.labels = self.data['step_timestamp'].apply(lambda x: 1 if x in [1, 2, 3] else 0).values  # Convert 1, 2, 3 to 1

    def load_data(self, folder_path):
        # Recursively load all .csv files from the folder and its subfolders
        all_files = glob.glob(os.path.join(folder_path, '**', '*.csv'), recursive=True)
        df_list = [pd.read_csv(file) for file in all_files]
        return pd.concat(df_list, ignore_index=True)

    def __len__(self):
        return len(self.data) - self.window_size + 1

    def __getitem__(self, idx):
        x = self.features[idx:idx + self.window_size]
        y = self.labels[idx + self.window_size - 1]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [15]:
# Parameters
folder_path = r'C:\Users\micha\Documents\magisterka\10stepstrain'
window_size = 160
batch_size = 32
test_split_ratio = 0.15  # 20% of the data for testing

# Create dataset
dataset = StepDataset(folder_path, window_size)

# Calculate the number of samples for training and testing
test_size = int(len(dataset) * test_split_ratio)
train_size = len(dataset) - test_size

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, 1)  # Binary classification

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Take the output of the last timestep
        out = self.fc(out)  # Logits
        return out
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model
model = LSTM(6, 64, 2, 0.2).to(device)

# Calculate the weight for the positive class (steps)
num_steps = sum(dataset.labels)  # Number of steps (1s)
num_no_steps = len(dataset.labels) - num_steps  # Number of no steps (0s)
pos_weight = torch.tensor([num_no_steps / num_steps], dtype=torch.float32).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 13

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Evaluation on test set
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels.float())
            test_loss += loss.item()
    test_loss /= len(test_loader)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

# Evaluate the model with leeway
model.eval()
correct = 0
total = 0

sampling_rate = 100  # Hz
time_leeway_ms = 100  # milliseconds
sample_leeway = int((time_leeway_ms / 1000) * sampling_rate)

real_steps = 0
counted_steps = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs).squeeze()
        predicted = (outputs > 0.5).float()

        labels_np = labels.cpu().numpy()
        predicted_np = predicted.cpu().numpy()

        for i in range(len(labels_np)):
            label = labels_np[i]
            pred = predicted_np[i]

            if label != 0:
                real_steps += 1
            if label != 0 and torch.any(predicted == label):
                counted_steps += 1
            if torch.any(predicted == label):
                correct += 1

            total += 1

try:
    accuracy = 100 * correct / total
    print(f'Accuracy with leeway: {accuracy:.2f}%')
    print(f'Real steps: {real_steps}, Counted steps: {counted_steps}')
except ZeroDivisionError:
    print('No steps detected in the test set')


Epoch 1/13, Train Loss: 0.5436, Test Loss: 0.5211
Epoch 2/13, Train Loss: 0.4393, Test Loss: 0.3700
Epoch 3/13, Train Loss: 0.3748, Test Loss: 0.3562
Epoch 4/13, Train Loss: 0.3459, Test Loss: 1.0008
Epoch 5/13, Train Loss: 0.3643, Test Loss: 0.7045
Epoch 6/13, Train Loss: 0.3491, Test Loss: 0.3203
Epoch 7/13, Train Loss: 0.3332, Test Loss: 0.3486
Epoch 8/13, Train Loss: 0.3244, Test Loss: 0.3918
Epoch 9/13, Train Loss: 0.3481, Test Loss: 0.3766
Epoch 10/13, Train Loss: 0.3405, Test Loss: 0.3366
Epoch 11/13, Train Loss: 0.3206, Test Loss: 0.4365
Epoch 12/13, Train Loss: 0.3261, Test Loss: 0.3456
Epoch 13/13, Train Loss: 0.3348, Test Loss: 0.3254
Accuracy with leeway: 100.00%
Real steps: 62, Counted steps: 62


In [ ]:
import torch
import pandas as pd
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time  # Import the time module

def test_single_file(model, csv_file, window_size, sample_leeway, output_file, device='cpu'):
    # Load the data
    data = pd.read_csv(csv_file)
    features = data[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']].values
    timestamps = data['step_timestamp'].values

    # Prepare the data
    inputs = []
    for i in range(len(features) - window_size + 1):
        x = features[i:i + window_size]
        inputs.append(torch.tensor(x, dtype=torch.float32))

    # Move model to the appropriate device
    model.to(device)
    model.eval()

    correct = 0
    total = 0
    predicted_steps = []

    # Measure inference time
    total_inference_time = 0.0

    # Start timing for the entire file
    file_start_time = time.time()

    with torch.no_grad():
        for i in range(len(inputs)):
            input_tensor = inputs[i].unsqueeze(0).to(device)  # Add batch dimension and move to device
            
            # Start timing for a single window
            start_time = time.time()
            
            # Perform inference
            output = model(input_tensor).squeeze()
            
            # End timing for a single window
            end_time = time.time()
            
            # Calculate inference time for the window
            inference_time = end_time - start_time
            total_inference_time += inference_time

            predicted = (output > 0.5).float()  # Use threshold to determine class (0 or 1)

            label = timestamps[i + window_size - 1]
            print(f'Predicted: {predicted}, Label: {label}')
            # Check if the prediction is within the leeway
            if abs(predicted.item() - label) <= sample_leeway:
                correct += 1
            predicted_steps.append(predicted.item())
            total += 1

    # End timing for the entire file
    file_end_time = time.time()
    file_inference_time = file_end_time - file_start_time

    if total != 0:
        accuracy = 100 * correct / total
        print(f'Accuracy with leeway: {accuracy}%')

    # # Calculate and print average inference time per window
    # avg_inference_time = total_inference_time / len(inputs)
    # print(f'Average inference time per window: {avg_inference_time:.6f} seconds')

    # # Print total inference time for the file
    # print(f'Total inference time for file: {file_inference_time:.6f} seconds')

    # Save the predicted steps to a CSV file
    predicted_steps_df = pd.DataFrame(predicted_steps, columns=['predicted_step_timestamp'])
    predicted_steps_df.to_csv(output_file, index=False)
    print(f'Predicted steps saved to {output_file}')

def process_folder(model, input_folder, output_folder, window_size, sample_leeway, device='cpu'):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over all files and subfolders in the input folder
    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith('.csv'):
                input_file = os.path.join(root, filename)
                relative_path = os.path.relpath(input_file, input_folder)
                output_file = os.path.join(output_folder, relative_path)
                os.makedirs(os.path.dirname(output_file), exist_ok=True)
                print(f'Processing {input_file}...')
                test_single_file(model, input_file, window_size, sample_leeway, output_file, device)

# Parameters
input_folder = r'C:\Users\micha\Documents\magisterka\Labeling\output_merged_test_train_split\test\first10'
output_folder = r'C:\Users\micha\Documents\magisterka\Labeling\predictions\train\specialized\first10'
window_size = 160
sampling_rate = 100  # Hz
time_leeway_ms = 100  # milliseconds
sample_leeway = int((time_leeway_ms / 1000) * sampling_rate)  # Convert ms to number of samples

# Initialize the model (make sure to load the trained model weights)
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, 1)  # Binary classification

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Take the output of the last timestep
        out = self.fc(out)  # Logits
        return out

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model
model = LSTM(6, 64, 2, 0.2).to(device)
model.load_state_dict(torch.load('LSTM_1CLASS_2DS_200wnd_13_pch_10steps.pth'))  # Load the trained model weights

# Process the folder
process_folder(model, input_folder, output_folder, window_size, sample_leeway, device)

C:\Users\micha\AppData\Local\Temp\ipykernel_15224\2703935542.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('LSTM_1CLASS_2DS_200wnd_1

Processing C:\Users\micha\Documents\magisterka\Labeling\output_merged_test_train_split\test\first10\combined_10steps_University_2nd_floor_3bb88968-d8ab-4df7-852b-8389748f3a30_accelerometer.csv...
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Predicted: 0.0, Label: 0
Pred

In [14]:
import pandas as pd

# Load the CSV file
input_file = r'C:\Users\micha\Documents\magisterka\Labeling\output_merged_test_train_split\test\first10\combined_eti10krokow4634c53c-0399-41a5-baa0-73475c39fc68_accelerometer.csv'
df = pd.read_csv(input_file)

# Drop the first column (by index)
df = df.drop(df.columns[0], axis=1)

# Save the modified DataFrame to a new file
output_file = 'combined_eti10krokow4634c53c-0399-41a5-baa0-73475c39fc68_accelerometer.cs'
df.to_csv(output_file, index=False)

print(f"First column removed. Saved to {output_file}")

First column removed. Saved to combined_eti10krokow4634c53c-0399-41a5-baa0-73475c39fc68_accelerometer.cs


In [ ]:
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile

# Load model and weights
model = LSTM(input_size=6, hidden_size=64, num_layers=2, dropout_prob=0.2)
model.load_state_dict(torch.load("LSTM_1CLASS_2DS.pth"))
model.eval()

# Dummy input to match your actual inference shape (e.g., batch_size=1, seq_len=160, features=6)
example_input = torch.rand(1, 160, 6)

# Trace the model
traced = torch.jit.trace(model, example_input)

# Optimize for mobile
optimized = optimize_for_mobile(traced)

# Save for Lite interpreter
optimized._save_for_lite_interpreter("LSTM_model.ptl")

C:\Users\micha\AppData\Local\Temp\ipykernel_15224\293121452.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("LSTM_1CLASS_2DS.pth"))
